<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# CeRAS life cycle assessment case study

This tutorial aims to generate the several environmental impacts of a CeRAS aircraft (based on report results, more information about the CeRAS reference aircraft configuration can be found [here](https://ceras.ilr.rwth-aachen.de/trac/wiki/CeRAS/AircraftDesigns/CSR01)). These impacts correspond to ReCiPe impacts category at both Midpoint and Endpoint levels.

This environmental study is conducted through the sizing of the CeRAS aircraft for a 2500NM mission range, and results are given for a longer range (2750NM) and a shorter one (500NM), for which a comparison is then made between both ranges.

***
***

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

## Imports

In [ ]:
import os.path as pth
import sys
import logging
import shutil

import openmdao.api as om

import fastoad.api as oad
from fastoad.io import VariableIO

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

sys.path.append(pth.abspath("."))
from ceras_utils import plot_mission_against_ceras

## Working directories and reference files
Here you can define your working directory and data storage folder as well as reference files names. 

In [ ]:
logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

DATA_FOLDER_PATH = "data"

WORK_FOLDER_PATH = "workdir"

SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "CeRAS_reference_data_lca.xml")

REFERENCE_FILE_FOR_PLOT = pth.join(DATA_FOLDER_PATH, "CeRAS_data_for_plots.xml")

CERAS_REFERENCE_RESULTS_FILE = pth.join(DATA_FOLDER_PATH, "CSR-01_missionDesign_R4630_PL17000_out.csv")

CERAS_SPP_DESIGN_REFERENCE_RESULTS_FILE = pth.join(DATA_FOLDER_PATH, "CSR-01_missionStudy_R5093_PL13608_out.csv")

CERAS_SPP_STUDY_REFERENCE_RESULTS_FILE = pth.join(DATA_FOLDER_PATH, "CSR-01_missionStudy_R926_PL13608_out.csv")

***
***

## 2. CeRAS reference case
Here, the CeRAS reference case is retrieved. 

The reference mission characteristics used for the sizing correspond to the *"MTOW mission"* defined in the CeRAS report whose specifications are reminded below:

* Range = 2500NM
* Diversion range = 200NM
* Payload = 17000kg
* Taxi in = 5min 
* Taxi out = 9min 
* Cruise Mach = 0.78

For the LCA purpose, only the main route segment is considered, therefore the considered range is 2500NM.

Once you create an adapted mission file you should generate or retrieve a configuration file that would allow for a mission analysis to be run. Let's have a look to such a [file](./data/operational_missions.yml): 

In [ ]:
CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, "operational_missions.yml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "oad_sizing_out.xml")
input_file = oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

## 3. Operational missions runs
### 3.1. Configuration

Now that the reference case has been retrieved, the simulation for the longer ans shorter ranges can be operated.
Their environmental analyzes are ccomputed with a set of assumptions:

* Load factor = 80% (120 passengers)
* Payload mass = 10886 kg
* Life duration of the aircraft = 25 years
* Number of cycles = 1198 for the 2750NM mission and 6590 for the 500NM mission

More information about the detailed LCA configuration can be found [here](https://arc.aiaa.org/doi/abs/10.2514/6.2022-1028). 

The `variable_viewer` can be used to change the range and the number of cycle to study both cases.

In [ ]:
oad.variable_viewer(input_file)

In [ ]:
SPP_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

The XML [output file](./workdir/operational_missions_out.xml) can be displayed in a table format using `variable_viewer`function.

In [ ]:
oad.variable_viewer(SPP_problem.output_file_path)

### 3.2. LCA results

#### 3.2..1. Climate Change first approximation - manual calculations
Based on the fuel consumption (OUTPUTS <i>data:mission:step:fuel_burn:total</i>) it is possible to have a first approximation regarding the Climate Change indicator. To do so, only carbon dioxide emissions are considered to decomplexify the problem. 

The final result need to be expressed per passenger.kilometer and needs to take into account the emissions from the fuel combustion but also the fuel production.

More precisely, you will need the following information:

* Fuel burn
* Range
* Number of passengers
* Carbon dioxide emission index
* Estimate the percentage that the fuel production represents (in terms of emissions) as compared to the fuel combustion

In [ ]:
#Manual calculations here

comb=15179.605696424993*3.15/(5093*120)
prod=0.15*comb
climate_change=comb+prod
print(climate_change,"kgCO2/pax.km")

#### 3.2.2. Midpoint

Now that the simulation is finished, it is possible to display the LCA results by plotting the Midpoint graph composed of 18 environmental indicators, and the contribution of each aircraft life cycle phase on these indicators. 

As each one of them has a specific unit, the plotted bar chart has been scaled and the results are express in percentage. However, the raw values can be seen by positioning the mouse on the bars.

In [ ]:
data_midpoint={"Indicators":["Agricultural Land Occupation (m2.a)", "Climate Change (kg CO2-eq)","Fossil Depletion (kg oil-eq)", 
                    "Freshwater Ecotoxicity (kg 1,4-DCB)", "Freshwater Eutrophication (kg P-eq)", "Human Toxicity (kg 1,4-DCB)",
                    "Ionizing Radiation (kBq U235-eq)","Marine Ecotoxicity (kg 1,4-DCB)","Marine Eutrophication (kg N-eq)", 
                    "Metal Depletion (kg Fe-eq)", "Natural Land Transformation (m2)", "Ozone Depletion (kg CFC-11)",
                    "Particulate Matter Formation (kg PM10-eq)", "Photochemical Oxidant Formation (kg NMVOC)", 
                    "Terrestrial Acidification (kg SO2-eq)","Terrestrial Ecotoxicity (kg 1,4-DCB)", 
                    "Urban Land Occupation (m2.a)", "Water Depletion (m3)"], 
               "Aircraft Production":SPP_problem["data:lca:midpoint:total:dataframe"][:,0],
               "Aircraft Production value":SPP_problem["data:lca:midpoint:total:dataframe"][:,0],
               "Airport":SPP_problem["data:lca:midpoint:total:dataframe"][:,1],
               "Airport value":SPP_problem["data:lca:midpoint:total:dataframe"][:,1],
               "Fuel Production":SPP_problem["data:lca:midpoint:total:dataframe"][:,2],
               "Fuel Production value":SPP_problem["data:lca:midpoint:total:dataframe"][:,2],
               "Aircraft Use":SPP_problem["data:lca:midpoint:total:dataframe"][:,3],
               "Aircraft Use value":SPP_problem["data:lca:midpoint:total:dataframe"][:,3],
               "Total":SPP_problem["data:lca:midpoint:total:dataframe"][:,4]}

midpoint_df=pd.DataFrame(data_midpoint)

fig_midpoint = px.bar(data_frame=midpoint_df, x="Indicators", 
                      y=["Aircraft Production","Airport","Fuel Production","Aircraft Use"],  width=1000, height=800,
                      color_discrete_sequence=['darkorange','gold','darkgray','royalblue'],barmode="relative",
                      title="<b>CeRAS Midpoint Impact Category results<b>", 
                      hover_data=[ "Aircraft Production value", "Airport value", "Fuel Production value", "Aircraft Use value",
                                  "Total"])
fig_midpoint.update_layout(template="simple_white", xaxis_title="Midpoint impact categories",xaxis_tickangle=-45,title_x=0.5,
                           yaxis_title="Percentage", barnorm="percent", font_family="Arial", title_font_size=20,
                           legend_title="<b>CeRAS LCA perimeters<b>", font_size=15,legend_font_family="Arial")
fig_midpoint.show()

#### 3.2.3. Endpoint

And finally the Endpoint results can be plotted. They are made of 17 indicators gathered in 3 main categories that are the Ecosystems Quality, the Human Health and the Resources. Every indicators are expressed in points, which make them directly comparable to one another within a same category.

In [ ]:
x = [
    ["Ecosystems Quality","Ecosystems Quality","Ecosystems Quality","Ecosystems Quality","Ecosystems Quality",
     "Ecosystems Quality","Ecosystems Quality","Ecosystems Quality", "Ecosystems Quality","Human Health", "Human Health",
     "Human Health","Human Health","Human Health","Human Health", "Resources", "Resources"],
    ["Terrestrial Acidification", "Terrestrial Ecotoxicity", "Agricultural Land Occupation", "Freshwater Eutrophication", 
     "Urban Land Occupation", "Freshwater Ecotoxicity", "Natural Land Transformation", "Marine Ecotoxicity",
     "Climate Change", "Human Toxicity", "Photochemical Oxidant Formation", "Ozone Depletion", "Particulate Matter Formation", 
     "Ionizing Radiation","Climate Change", "Fossil Depletion", "Metal Depletion"]]  

fig_endpoint= go.Figure()
fig_endpoint.add_bar(x=x, y=SPP_problem["data:lca:endpoint:total:dataframe"][:,0], name="Aircraft Production")
fig_endpoint.add_bar(x=x, y=SPP_problem["data:lca:endpoint:total:dataframe"][:,1], name="Airport")
fig_endpoint.add_bar(x=x, y=SPP_problem["data:lca:endpoint:total:dataframe"][:,2], name="Fuel Production")
fig_endpoint.add_bar(x=x, y=SPP_problem["data:lca:endpoint:total:dataframe"][:,3], name="Aircraft Use")
fig_endpoint.update_layout(barmode="relative", xaxis_title="Endpoint impact categories", yaxis_title="Points",
                           title="<b>CeRAS Endpoint Impact Category results<b>", template="simple_white", 
                           font_family="Arial", font_size=15, title_font_size=20, title_x=0.5,
                           legend_title="<b>CeRAS LCA perimeters<b>", width=1000, height=800, 
                           colorway=['darkorange', 'gold', 'darkgray', 'royalblue'], 
                           hoverlabel=dict(bgcolor="blue", font_size=12, font_family="Arial"),legend_font_family="Arial")
fig_endpoint.show()

### 3.3. Range comparison
It is now possible to compare the results obtained for the previous range with a shorter range of 500NM. 

First, the results for this latter need to be retrieved.

In [ ]:
RANGE_500NM_FILE = pth.join(DATA_FOLDER_PATH, "operational_missions_out_500NM.xml")

variable = VariableIO(RANGE_500NM_FILE, None).read()

midpoint_500 = variable["data:lca:midpoint:total:dataframe"].value
endpoint_500 = variable["data:lca:endpoint:total:dataframe"].value
Total_midpoint_500 = np.array(midpoint_500)
Total_endpoint_500 = np.array(endpoint_500)

Then, the comparison can be ploted. 

For all midpoint indicators, the results have been normalized with the results obtained for the 2750NM range, considered as the reference case. 

This way, the results are shown in percent and the difference between bars of the same environmental indicators represents the relative difference between both ranges.

In [ ]:
norm_2750 = SPP_problem["data:lca:midpoint:total:dataframe"][:,-1]/SPP_problem["data:lca:midpoint:total:dataframe"][:,-1]*100
norm_500 = Total_midpoint_500[:,-1]/SPP_problem["data:lca:midpoint:total:dataframe"][:,-1]*100

Range={"Indicators":["Agricultural Land Occupation (m2.a)", "Climate Change (kg CO2-eq)","Fossil Depletion (kg oil-eq)", 
                    "Freshwater Ecotoxicity (kg 1,4-DCB)", "Freshwater Eutrophication (kg P-eq)", "Human Toxicity (kg 1,4-DCB)",
                    "Ionizing Radiation (kBq U235-eq)","Marine Ecotoxicity (kg 1,4-DCB)","Marine Eutrophication (kg N-eq)", 
                    "Metal Depletion (kg Fe-eq)", "Natural Land Transformation (m2)", "Ozone Depletion (kg CFC-11)",
                    "Particulate Matter Formation (kg PM10-eq)", "Photochemical Oxidant Formation (kg NMVOC)", 
                    "Terrestrial Acidification (kg SO2-eq)","Terrestrial Ecotoxicity (kg 1,4-DCB)", 
                    "Urban Land Occupation (m2.a)", "Water Depletion (m3)"], "2750NM":norm_2750, "500NM":norm_500, 
       "2750NM_total":SPP_problem["data:lca:midpoint:total:dataframe"][:,-1], "500NM_total": Total_midpoint_500[:,-1]}

Range_df=pd.DataFrame(Range)

fig_range=px.bar(data_frame=Range_df, x="Indicators", y=["2750NM","500NM"], color_discrete_sequence=["royalblue","firebrick"], 
                 barmode="group", title="<b>CeRAS Midpoint impacts category - Range case study<b>", 
                 hover_data=["2750NM_total", "500NM_total"], width=1000, height=800)
fig_range.update_layout(template="simple_white",xaxis_title="Midpoint Impacts",xaxis_tickangle=-45, yaxis_title="Percentage", 
                          font_family="Arial", font_size=15, title_font_size=20, title_x=0.5, 
                         legend_title="<b>Range<b>", legend_font_family="Arial")
fig_range.show()
